In [1]:
# !jupyter nbextension enable --py gmaps

In [2]:
# !pip install gmaps

In [3]:
# Dependencies
import requests
import json
import pandas as pd
import numpy as np
import time
import gmaps

from pprint import pprint

# Google developer API key
from config import gkey

#pd.options.mode.chained_assignment = None 

# Read in Arlington County Housing Assessment data
arl_data = pd.read_csv("2018_Arl_Asmt.csv")
arl_data.head()

,IvwPropertyAssessmentHistoryKey,ProvalLrsnId,RealEstatePropertyCode,MasterRealEstatePropertyCode,ReasPropertyStatusCode,PropertyClassTypeCode,PropertyClassTypeDsc,PropertyStreetNbrNameText,PropertyStreetNbr,PropertyStreetNbrSuffixCode,...,PropertyExpiredInd,CommercialInd,DistrictNbr,TaxExemptionTypeDsc,CondominiumProjectName,TaxBalanceAmt,AssessedYearDate,TotalAssessedAmt,AssessmentDate,AssessmentChangeReasonTypeDsc
0,36,133,1001006,1001006,T,510,510-Res - Vacant(SF & Twnhse),N ROCHESTER ST,NaN,NaN,...,False,False,0G,NaN,NaN,0.0,2018,16600,1/1/2018 0:00,01- Annual
1,73,134,1001007,1001007,A,511,511-Single Family Detached,3007 N ROCHESTER ST,3007.0,NaN,...,False,False,0G,NaN,NaN,0.0,2018,1772500,1/1/2018 0:00,01- Annual
2,110,136,1001009,1001009,A,511,511-Single Family Detached,6547 WILLIAMSBURG BLVD,6547.0,NaN,...,False,False,0G,NaN,NaN,0.0,2018,773000,1/1/2018 0:00,01- Annual
3,147,137,1001010,1001010,A,511,511-Single Family Detached,6541 WILLIAMSBURG BLVD,6541.0,NaN,...,False,False,0G,NaN,NaN,0.0,2018,991300,1/1/2018 0:00,01- Annual
4,184,139,1001012,1001012,A,511,511-Single Family Detached,3518 N SOMERSET ST,3518.0,NaN,...,False,False,0G,NaN,NaN,0.0,2018,1341700,1/1/2018 0:00,01- Annual


In [4]:
short_list = arl_data[arl_data["PropertyClassTypeCode"].isin([511,512,513,514,515,611,612,613,616,630])] 
short_list = short_list[short_list['AssessmentChangeReasonTypeDsc'].isin(['01- Annual'])]
short_list.drop(short_list[short_list['CommercialInd'] == True].index, inplace=True)
short_list = short_list[pd.isnull(short_list['TaxExemptionTypeDsc'])]
short_list = short_list[short_list['TotalAssessedAmt'] > 70000]  
short_list['State']='VA'
short_list["Lat"] = ""
short_list["Lng"] = ""

# Confirm short list is correct  
short_list.head()

,IvwPropertyAssessmentHistoryKey,ProvalLrsnId,RealEstatePropertyCode,MasterRealEstatePropertyCode,ReasPropertyStatusCode,PropertyClassTypeCode,PropertyClassTypeDsc,PropertyStreetNbrNameText,PropertyStreetNbr,PropertyStreetNbrSuffixCode,...,TaxExemptionTypeDsc,CondominiumProjectName,TaxBalanceAmt,AssessedYearDate,TotalAssessedAmt,AssessmentDate,AssessmentChangeReasonTypeDsc,State,Lat,Lng
1,73,134,1001007,1001007,A,511,511-Single Family Detached,3007 N ROCHESTER ST,3007.0,NaN,...,NaN,NaN,0.0,2018,1772500,1/1/2018 0:00,01- Annual,VA,,
2,110,136,1001009,1001009,A,511,511-Single Family Detached,6547 WILLIAMSBURG BLVD,6547.0,NaN,...,NaN,NaN,0.0,2018,773000,1/1/2018 0:00,01- Annual,VA,,
3,147,137,1001010,1001010,A,511,511-Single Family Detached,6541 WILLIAMSBURG BLVD,6541.0,NaN,...,NaN,NaN,0.0,2018,991300,1/1/2018 0:00,01- Annual,VA,,
4,184,139,1001012,1001012,A,511,511-Single Family Detached,3518 N SOMERSET ST,3518.0,NaN,...,NaN,NaN,0.0,2018,1341700,1/1/2018 0:00,01- Annual,VA,,
5,221,140,1001013,1001013,A,511,511-Single Family Detached,3526 N SOMERSET ST,3526.0,NaN,...,NaN,NaN,0.0,2018,732100,1/1/2018 0:00,01- Annual,VA,,


In [5]:
params = {"key": gkey}

try:
    for index, row in short_list[0:100].iterrows():
        base_url = "https://maps.googleapis.com/maps/api/geocode/json"

        street = row["PropertyStreetNbrNameText"]
        city = row["PropertyCityName"]
        state = row["State"]
        zip = row["PropertyZipCode"]

        params['address'] = f"{street},{city},{state},{zip}"

        houses_lat_lng = json.loads(requests.get(base_url, params=params).content)

        current_lat = houses_lat_lng["results"][0]["geometry"]["location"]["lat"]
        current_lng = houses_lat_lng["results"][0]["geometry"]["location"]["lng"]

        short_list.loc[index, "Lat"] = current_lat
        short_list.loc[index, "Lng"] = current_lng

        print(short_list.loc[index, "Lat"])
        print(short_list.loc[index, "Lng"])
        
        time.sleep(.02)
        
except KeyError:
    print("NaN")
    
    
house_list = short_list[["PropertyStreetNbrNameText",
                        "PropertyCityName", "State", "PropertyZipCode",
                        "TotalAssessedAmt", "Lat", "Lng"]]

house_list.head(100)

38.8998429
-77.1601325
38.8997628
-77.1594135
38.8998984
-77.1591888
38.90026960000001
-77.1596382
38.9002645
-77.15995269999999
38.9002142
-77.1602223
38.9001965
-77.1606043
38.900465
-77.1606717
38.90060039999999
-77.16080649999999
38.9008162
-77.16094129999999
38.9012201
-77.16114350000001
38.9014032
-77.1611659
38.9016341
-77.1610761
38.9019729
-77.1606941
38.902126
-77.16044699999999
38.9024422
-77.1600426
38.90267559999999
-77.1597954
38.900462
-77.161233
38.9005439
-77.16129099999999
38.900613
-77.1613349
38.900681
-77.161378
38.90075
-77.16142200000002
38.900819
-77.16146599999999
38.900904
-77.161516
38.901097
-77.161796
38.90109
-77.161896
38.9010184
-77.1620317
38.9009566
-77.1621242
38.900734
-77.162391
38.900657
-77.162398
38.900396
-77.162182
38.9003749
-77.16206900000002
38.900346
-77.161875
38.9003255
-77.1617691
38.9002991
-77.1616469
38.9020961
-77.1598179
38.90223659999999
-77.1596382
38.9025479
-77.1591888
38.9023171
-77.1589192
38.9000893
-77.15838
38.900044
-77.15

,PropertyStreetNbrNameText,PropertyCityName,State,PropertyZipCode,TotalAssessedAmt,Lat,Lng
1,3007 N ROCHESTER ST,ARLINGTON,VA,22213,1772500,38.8998,-77.1601
2,6547 WILLIAMSBURG BLVD,ARLINGTON,VA,22213,773000,38.8998,-77.1594
3,6541 WILLIAMSBURG BLVD,ARLINGTON,VA,22213,991300,38.8999,-77.1592
4,3518 N SOMERSET ST,ARLINGTON,VA,22213,1341700,38.9003,-77.1596
5,3526 N SOMERSET ST,ARLINGTON,VA,22213,732100,38.9003,-77.16
6,3530 N SOMERSET ST,ARLINGTON,VA,22213,1168100,38.9002,-77.1602
7,3534 N SOMERSET ST,ARLINGTON,VA,22213,2312800,38.9002,-77.1606
8,3538 N SOMERSET ST,ARLINGTON,VA,22213,752400,38.9005,-77.1607
9,3544 N SOMERSET ST,ARLINGTON,VA,22213,776300,38.9006,-77.1608
10,3550 N SOMERSET ST,ARLINGTON,VA,22213,761000,38.9008,-77.1609


In [7]:
metro_list = pd.DataFrame(
   {'metro': ['Arlington Cemetery','Ballston MU','Clarendon','Court House','Crystal City','East Falls Church','Pentagon','Pentagon City','Ronald Reagan Washington National Airport','Rosslyn','Virginia Square GMU'],
    'street': ['1000 North Memorial Drive', '4230 Fairfax Drive', '3100 Wilson Blvd', '2100 Wilson Blvd', '1750 South Clark St.', '2001 N. Sycamore St.', '2 South Rotary Road', '1250 South Hayes St.', '2400 S. Smith Blvd.', '1850 N. Moore Street', '3600 Fairfax Drive'],
    'city': ['Arlington', 'Arlington', 'Arlington', 'Arlington', 'Arlington', 'Arlington', 'Arlington', 'Arlington', 'Arlington', 'Arlington', 'Arlington'],
    'state': ['VA', 'VA', 'VA', 'VA', 'VA', 'VA', 'VA', 'VA', 'VA', 'VA', 'VA'],
    'zip': ['22211', '22201', '22201', '22201', '22202', '22205', '22202', '22202', '22202', '22209', '22201']
   })

metro_list["Lat"] = ""
metro_list["Lng"] = ""

params = {"key": gkey}

for index, row in metro_list.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    street = row['street']
    city = row['city']
    state = row['state']
    zip = row['zip']

    params['address'] = f"{street},{city},{state},{zip}"

    metros_lat_lng = requests.get(base_url, params=params)
    
    metros_lat_lng = metros_lat_lng.json()

    metro_list.loc[index, "Lat"] = metros_lat_lng["results"][0]["geometry"]["location"]["lat"]
    metro_list.loc[index, "Lng"] = metros_lat_lng["results"][0]["geometry"]["location"]["lng"]

metro_list.head(15)

,metro,street,city,state,zip,Lat,Lng
0,Arlington Cemetery,1000 North Memorial Drive,Arlington,VA,22211,38.8846,-77.0632
1,Ballston MU,4230 Fairfax Drive,Arlington,VA,22201,38.882,-77.1115
2,Clarendon,3100 Wilson Blvd,Arlington,VA,22201,38.8871,-77.0952
3,Court House,2100 Wilson Blvd,Arlington,VA,22201,38.8916,-77.0849
4,Crystal City,1750 South Clark St.,Arlington,VA,22202,38.8572,-77.0519
5,East Falls Church,2001 N. Sycamore St.,Arlington,VA,22205,38.8855,-77.1563
6,Pentagon,2 South Rotary Road,Arlington,VA,22202,38.871,-77.0559
7,Pentagon City,1250 South Hayes St.,Arlington,VA,22202,38.8631,-77.0601
8,Ronald Reagan Washington National Airport,2400 S. Smith Blvd.,Arlington,VA,22202,38.8497,-77.0435
9,Rosslyn,1850 N. Moore Street,Arlington,VA,22209,38.8969,-77.072


In [8]:
# Adding layers to the map
coordinates = metro_list[["Lat", "Lng"]]

# Customize the size of the figure
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
metro_fig = gmaps.figure(layout=figure_layout)

# Assign the marker layer to a variable
markers = gmaps.marker_layer(coordinates)

# Add the layer to the map
metro_fig.add_layer(markers)

metro_fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

In [9]:
mapping_house_list = house_list.loc[(house_list["Lat"] != '') | (house_list["Lng"] != '')] 

mapping_house_list["Lat"] = mapping_house_list["Lat"].astype("float")
mapping_house_list["Lng"] = mapping_house_list["Lng"].astype("float")  
mapping_house_list["TotalAssessedAmt"] = mapping_house_list["TotalAssessedAmt"].astype("float")  

locations = mapping_house_list[["Lat", "Lng"]]

assmt = mapping_house_list["TotalAssessedAmt"]

fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=assmt,
                                 dissipating=True, max_intensity=5,
                                 point_radius=5)

# Add layer
fig.add_layer(heat_layer)
fig.add_layer(markers)

# Display figure
fig

C:\Users\obrie\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\obrie\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\obrie\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

Figure(layout=FigureLayout(height='420px'))

In [10]:
# BONUS
# Plost as a Hybrid map
fig = gmaps.figure(map_type="HYBRID")

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=assmt, 
                                 dissipating=True, max_intensity=5,
                                 point_radius=1)

fig.add_layer(heat_layer)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))